# Model run information 

In [ ]:
import pandas as pd
import importlib
import re
import warnings
import calendar
import copy
import oceanval
ff  = "../../matched/definitions.pkl"
import pickle
with open(ff, "rb") as f:
    definitions = pickle.load(f)

import glob
import geopandas as gpd
import jellyfish
import nctoolkit as nc
nc.options(parallel=True)
import os
import pickle
import numpy as np
import xarray as xr
from IPython.display import Markdown as md_markdown
import cmocean as cm
from tqdm import tqdm
import hvplot.pandas
from plotnine import *
from IPython.core.interactiveshell import InteractiveShell
from oceanval.tidiers import tidy_info
InteractiveShell.ast_node_interactivity = "all"

try:
    lon_lim = None
    lat_lim = None
except:
    lon_lim = None
    lat_lim = None
if lon_lim is None:
    lon_lim = [-180, 180]
if lat_lim is None:
    lat_lim = [-90, 90]

try:
    concise = False
except:
    concise = False

# in one line
from oceanval.tidiers import fix_basename, fix_unit, df_display, tidy_summary_paths, md, md_basic

warnings.filterwarnings('ignore')

%load_ext rpy2.ipython

test_status = False


i_figure = 1
i_table = 1
stamp = nc.session_info["stamp"]
out = ".trackers/" + stamp
fast_plot = False
if not os.path.exists(".trackers"):
    os.makedirs(".trackers")
# save out as empty file
with open(out, 'w') as f:
    f.write("")
nws = False

def bin_value(x, bin_res):
    return np.floor((x + bin_res / 2) / bin_res + 0.5) * bin_res - bin_res / 2

try:
    vv_name = variable
except:
    vv_name = "summary"
    variable = "summary"
Variable = variable.title()
if vv_name in ["benbio"]:
    compact = True
try:
    vv_name = definitions[variable].long_name 
except:
    pass
import glob

In [ ]:
# open pickle
model_label = "ersem"
with open("../../matched/times_dict.pkl", "rb") as f:
    times_dict = pickle.load(f)
    ff = list(times_dict.keys())[0]
ds = nc.open_data(ff, checks = False)
amm7 = False
if ds.contents.npoints.max() == 111375:
    if ds.contents.nlevels.max() == 51:
        amm7 = True
        model_label = "ersem"

import pickle
try:
    times_dict = pickle.load(open("../../matched/times_dict.pkl", "rb"))
    df_times = pd.concat(times_dict.values()).drop_duplicates().sort_values(["year", "month", "day"])
except:
    pass

In [ ]:
def ordinal(n: int):
    if 11 <= (n % 100) <= 13:
        suffix = 'th'
    else:
        suffix = ['th', 'st', 'nd', 'rd', 'th'][min(n % 10, 4)]
    return str(n) + suffix

In [ ]:
from datetime import date
def get_date(start):
    day = start.day
    month = start.month
    year = start.year
    tidied =  date(day = day, month = month, year = year).strftime('%d %B %Y')
    tidied = tidied.split(" ")
    tidied[0] = ordinal(int(tidied[0]))
    # add of
    tidied.insert(1, "of")
    return " ".join(tidied)
    
try:
    start = get_date(df_times.iloc[0])
    end = get_date(df_times.iloc[-1])
except:
    pass

In [ ]:
%%capture --no-display
plot_bath = False
lon_min = None
lon_max = None
lat_min = None
lat_max = None
if True:
    if os.path.exists(f"../../matched/model_bathymetry.nc"):
        ds_bath = nc.open_data(f"../../matched/model_bathymetry.nc")
        try:
            ds_bath.fix_amm7_grid()
        except:
            pass
        ds_bath.rename({ds_bath.variables[0]:"Bathymetry"})
        ds_bath.set_longnames({"Bathymetry":"Bathymetry"})
        ds_bath.set_units({"Bathymetry":"m"})
        try:
            ds_bath.pub_plot(colours = cm.cm.deep, land = "grey")
            plot_bath = True
        except:
            ds_plot = ds_bath.copy()
            lon_name = [x for x in ds_plot.to_xarray().coords if "lon" in x][0]
            lat_name = [x for x in ds_plot.to_xarray().coords if "lat" in x][0]
            lons = ds_plot.to_xarray()[lon_name].values
            # flatten
            lons = list(set(lons.flatten()))
            lons.sort()
            lon_res = np.abs(lons[0] - lons[1])
            # do the same for lats
            lats = ds_plot.to_xarray()[lat_name].values
            # flatten
            lats = list(set(lats.flatten()))
            lats.sort()
            lat_res = np.abs(lats[0] - lats[1])
            lon_min = np.min(lons)
            lon_max = np.max(lons)
            lat_max = np.max(lats)
            lat_min = np.min(lats) 
            # coerce to float
            lon_min = float(lon_min)
            lon_max = float(lon_max)
            lat_min = float(lat_min)
            lat_max = float(lat_max)

            # regrid
            ds_plot.to_latlon(lon = [lon_min , lon_max], lat = [lat_min, lat_max], res = [lon_res, lat_res]) 
            ds_plot.run()
            ds_plot.pub_plot(colours = cm.cm.deep, land = "grey")
            plot_bath = True




In [ ]:
if plot_bath:
    if True:
        extent = oceanval.utils.get_extent(ds_bath[0])
        lon_min = extent[0]
        lon_max = extent[1]
        lat_min = extent[2]
        lat_max = extent[3]
        # round to 3 dp
        lon_min = round(lon_min, 3)
        lon_max = round(lon_max, 3)
        lat_min = round(lat_min, 3)
        lat_max = round(lat_max, 3)

In [ ]:
if plot_bath:
    if amm7:
        lon_min = -20
        lon_max = 13
        lat_min = 40
        lat_max = 65
    md(f"**Figure {i_figure}**: Bathymetry of the model domain. The model domain extends from {lon_min}° to {lon_max}° longitude and {lat_min}° to {lat_max}° latitude.")
    i_figure += 1